<a href="https://colab.research.google.com/github/JorgeGarcia2/FSMGenerator/blob/main/FixDescription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importar las librerias necesarias

In [ ]:
import os
import pandas as pd
from openpyxl import load_workbook
import re


# Funciones
Cargar las funciones

In [ ]:
def getFileName():
    fileCont = True
    # Check if the user wants to continue trying new file names
    while(fileCont):
        fileCont = False
        filePath = "/wis/" + input("\nWhich WI do you want to use? ") + "/"
        
        if(os.path.isdir(filePath)):
            # Get files in the directory
            for file in os.listdir(filePath):

                if(re.search(r"^\w*\.xlsm$", file)):
                    fileName = file
                    # Ask user if he wants to use the file
                    t = input("\n XLSM file found, do you want to use: "+fileName+" (Y,N)? ")
                    
                    if t!="n" and t!="N":

                        # If it's going to be used, fill fileCont with a non-empty string
                        print(f"\nFile {fileName} will be used!\n\n")
                        fileCont = True
                        break
                    else:
                        # else: fileName = "./"
                        print("File not used!")
        else:
            print("The specified path was not found")
        
        # If the program does not find the file, ask to try again
        if (fileCont == False):
            t = input("\nDo you want to try again (Y/N)? ")
            if (t == "N" or t == "n"):
                fileName = ""
                fileCont = False
            else: fileCont = True
        else: fileCont = False
    return fileName, filePath



In [ ]:
def getDescription(fullPath):
    description = ""
    report =""           

    book = load_workbook(fullPath + ".xlsm", keep_vba=True)
    excelFile = pd.read_excel(fullPath + ".xlsm", engine="openpyxl", sheet_name=None)
    sheet2Name = list(excelFile.keys())[1]
    for nRow in  range(1, len(excelFile[sheet2Name].index)):
        description = excelFile[sheet2Name].iat[nRow,4]
        report += f"{' '*30} Descripcion {nRow}\n"
        report += "-"*100 + "\n"
        report += excelFile[sheet2Name].iat[nRow,4] + "\n"
        report += "-"*100 + "\n"
        newDescription = fixDescription(description)
        book.worksheets[1].cell(2+nRow,5).value = newDescription
        report += newDescription + "\n"
        report += "_"*100 + "\n"
    fixedFile = fullPath + "_fixed.xlsm"
    book.save(fixedFile)
    print(f"\nSUCCESS: Fixed file {fixedFile} created")

    return report

In [35]:
def fixDescription(desc):
    regexLines = r"(((o|-|•|\*)\s{1})*\w+.*)" 
    # Formar lista de lineas no vacias
    linest = re.findall(regexLines, desc)
    lines = []
    for line in linest:
        if line[0] != '_x000d_':
            lines.append(list(line))
    
    newDesc = ""
    for nLine in range(len(lines)):

        #lines[nLine][0] = re.sub(r"((^(o|-|•|\*)\s{1}))", ". ", lines[nLine][0]) # Sustituir viñetas "o, -, *, •" por ". "
        IsStep = re.search(r"^\d", lines[nLine][0]) # Determina si se trata de un Step
        # Si la primera linea es un step, agrega un header por default y concatena en la siguiente linea el step
        if (nLine == 0): 
            if (IsStep):
                #lines[nLine] = re.sub(r"(\s*>\s*)", "-", lines[nLine]) # Quitar caracteres especiales
                newDesc += "Pasos,\n" + lines[nLine][0]
            # De lo contrario deja el Header en la primera linea
            else:   
                    newDesc += lines[nLine][0]
                    #newDesc += "Pasos,\n1 " + lines[nLine][0]

        elif(IsStep):
            newDesc += "\n" + lines[nLine][0]
        else: newDesc += ", " + lines[nLine][0]

    return newDesc

Main

In [37]:
fileName, filePath = getFileName()
if fileName != "":
    fileName_NE = fileName.split(".")[0]
    textReport = getDescription(filePath + fileName_NE)
else:
    print("No File, exiting the program")


Which WI do you want to use? 3285

 XLSM file found, do you want to use: PR413_MD8_D3285_PruebasdeSistema_Ciclo01.xlsm (Y,N)? 

File PR413_MD8_D3285_PruebasdeSistema_Ciclo01.xlsm will be used!




/usr/local/lib/python3.7/dist-packages/openpyxl/reader/worksheet.py:322: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)



SUCCESS: Fixed file /wis/3285/PR413_MD8_D3285_PruebasdeSistema_Ciclo01_fixed.xlsm created


/usr/local/lib/python3.7/dist-packages/openpyxl/comments/shape_writer.py:75: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if not shape_types:
